In [2]:
import warnings
import os
warnings.filterwarnings("ignore")

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Usuing device: {device}')

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer
from transformers import TextDataset, DataCollatorForLanguageModeling

Usuing device: cuda


In [24]:
def contar_caracteres(archivo, max_block_size=128):
    with open(archivo, 'r', encoding='utf-8') as file:
        text = file.read()
    
    word_count = len(text.split())
    block_size = min(word_count, max_block_size)
    return block_size

def load_dataset(file_path, tokenizer):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = contar_caracteres(file_path),
        cache_dir="C:/Users/User/TP1/Cache",
    )
    print(contar_caracteres(file_path))
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

In [25]:
def train(train_file_path,model_name,output_dir,overwrite_output_dir,per_device_train_batch_size,num_train_epochs):
  
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          logging_dir="C:/Users/User/TP1/logs",
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

In [26]:
## GPT-2 Small ('gpt2'): 124 million parameters.
## GPT-2 Medium ('gpt2-medium'): 345 million parameters.
## GPT-2 Large ('gpt2-large'): 774 million parameters.
## GPT-2 XL ('gpt2-xl'): 1.5 billion parameters.

label = 1
model_name = f'Models/Label {label}'
output_dir = f'C:/Users/User/TP1/Models/Label {label}'
overwrite_output_dir = True
per_device_train_batch_size = 3
num_train_epochs = 1.0
save_steps = 50000

In [27]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.save_pretrained(output_dir)
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model.save_pretrained(output_dir)

In [ ]:
Bloque = 1
path = f"Data/Dato_Bloques/Label {label}/Bloque {Bloque}/Code"
archivos = [archivo for archivo in os.listdir(path)]
for archivo in archivos:
    print(archivo)
    archivos_txt = [archivo for archivo in os.listdir(f"{path}/{archivo}") if archivo.endswith('.txt')]
    for archivo_txt in archivos_txt:
        if contar_caracteres(f"{path}/{archivo}/{archivo_txt}") != 0:
            print(archivo_txt)
            train(f"{path}/{archivo}/{archivo_txt}",model_name,output_dir,overwrite_output_dir,per_device_train_batch_size,num_train_epochs)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
# Herramientas de limpieza pyright, python lsv server
tokenizer = GPT2Tokenizer.from_pretrained('D:/Models/Model/Label 0')
model = GPT2LMHeadModel.from_pretrained('D:/Models/Model/Label 0')

def generate(code, max_length=300):
    tokenized = tokenizer.encode(code, return_tensors='pt')
    resp = model.generate(
        tokenized,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )

    return (tokenizer.decode(resp[0]))

print(generate("from"))

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X[:,0])


Y_train=to_categorical(Y_train)


def train_transform_data(df):
    for col in df.columns:
        if df[col].dtype in ('O') and df[col].dtype == 'object':
            le.fit(df[col])
             df[col] = le.transform(df[col])
          return df
    def test_transform_transform_data(df):
            if le.fit(df[col].dtype in ('O') and df[col].dtype == 'object':
                            and le.transform(df[col)!= 'cat':
                                   


In [3]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from Levenshtein import distance
from rouge_score import rouge_scorer

import sys
sys.path.append("C:/Users/User/CodeXGLUE")  # Asegúrate de que este camino apunte al directorio que contiene codebleu.

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Métricas Automáticas
## Perplexidad:

Mide cuán bien el modelo predice una muestra de texto. Una perplexidad más baja indica que el modelo es mejor para predecir el siguiente token en una secuencia. Principalmente durante el entrenamiento para monitorear el progreso.

## BLEU (Bilingual Evaluation Understudy):

Mide la similitud entre el código generado y uno o más códigos de referencia, basándose en la coincidencia de n-gramas. Comúnmente utilizado en la generación de texto y código para evaluar la calidad de las secuencias generadas.

## METEOR (Metric for Evaluation of Translation with Explicit ORdering):

Similar a BLEU pero considera la coincidencia de sinónimos y la coincidencia flexible de palabras. Evalúa la calidad del texto generado con mayor sensibilidad a la elección de palabras.

## ROUGE (Recall-Oriented Understudy for Gisting Evaluation):

Mide la superposición de n-gramas, así como la superposición de palabras y la longitud de las secuencias entre el texto generado y las referencias. Frecuentemente utilizado para evaluar resúmenes automáticos y generación de texto.

# Métricas Específicas para Generación de Código
## Exact Match (EM):

Mide si el código generado es exactamente igual al código de referencia. Útil para casos donde la exactitud completa es crítica.

## CodeBLEU:

Métrica especializada para evaluar la calidad del código generado, teniendo en cuenta aspectos específicos del código como la sintaxis, la estructura y la lógica. Especialmente útil para la evaluación de modelos de generación de código.


In [4]:
Label = 17

In [8]:
# New
input_csv = f'Data/Dato_Bloques/Label {Label}/archivo_1_promts.csv'
df = pd.read_csv(input_csv)
df['generated_text'] = pd.Series(dtype=object)
df['Exact Match'] = pd.Series(dtype=float)
df['Perplexity'] = pd.Series(dtype=float)
df['Bleu'] = pd.Series(dtype=float)
df['Meteor'] = pd.Series(dtype=float)
df['Rouge1'] = pd.Series(dtype=float)
df['Rouge2'] = pd.Series(dtype=float)
df['RougeL'] = pd.Series(dtype=float)

In [ ]:
# Old
input_csv = f'D:/Models/Logs/Label {Label}/Puntaje.csv'
df = pd.read_csv(input_csv)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(f'D:/Models/Model/Label {Label}')
model = GPT2LMHeadModel.from_pretrained(f'D:/Models/Model/Label {Label}')

model.eval()

# Evaluar segun la tarea
def generate_text(prompt, max_length=80): # Limit 1064
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def calculate_similarity(reference, candidate):
    max_length = max(len(reference), len(candidate))
    similarity = (max_length - distance(reference, candidate)) / max_length * 100
    return similarity

def calculate_Perplexity(encodings):
    with torch.no_grad():
        outputs = model(**encodings, labels=encodings['input_ids'])
        loss = outputs.loss
        perplexity = torch.exp(loss)
    
    return perplexity.item()

def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores

def adjust_reference(reference, target_length):
    reference_tokens = reference.split()
    if len(reference_tokens) > target_length:
        return ' '.join(reference_tokens[:target_length])
    else:
        return reference + ' ' * (target_length - len(reference_tokens))
    
for index, row in df.iterrows():
    if pd.isna(df['generated_text'][index]) and (900 - len(df['promts'][index])) < 1024:
        prompt = row['promts']
        reference = row['code_block']

        generated_text = generate_text(prompt, (900 - len(prompt)))
        reference = adjust_reference(reference, len(generated_text.split()))

        encodings = tokenizer(generated_text, return_tensors='pt')
    
        accuracy = calculate_similarity(reference, generated_text)
        Perplexity = calculate_Perplexity(encodings)
        
        reference_tokens = nltk.word_tokenize(reference)
        candidate_tokens = nltk.word_tokenize(generated_text)

        bleu = sentence_bleu([reference_tokens], candidate_tokens)
        meteor = meteor_score([reference_tokens], candidate_tokens)
        rouge_scores = calculate_rouge(reference, generated_text)

        df['generated_text'][index] = generated_text
        df['Exact Match'][index] = accuracy
        df['Perplexity'][index] = Perplexity
        df['Bleu'][index] = bleu
        df['Meteor'][index] = meteor
        df['Rouge1'][index] = rouge_scores['rouge1'].fmeasure
        df['Rouge2'][index] = rouge_scores['rouge2'].fmeasure
        df['RougeL'][index] = rouge_scores['rougeL'].fmeasure

        print(f"{index}:  Exact Match:{accuracy}  Perplexity:{Perplexity}   bleu:{bleu}   meteor:{meteor}   Rouge:{rouge_scores}")

        if index % 20 == 0:
            df.to_csv(f'D:/Models/Logs/Label {Label}/Puntaje.csv', index=False)

In [9]:
df.head(20) # 1000 cada uno - 5 mejores - testearlo 

,code_block,promts,generated_text,Exact Match,Perplexity,Bleu,Meteor,Rouge1,Rouge2,RougeL
0,reduce_features_df.drop(columns=['days_since_p...,reduce_features_df.drop(columns=['days_since_p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,X = df.drop(columns=['label'])\nX.head(),X = df.drop(columns=['label'])\nX.head(),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"image_study_df = image_study_df[[""image_id"", ""...","image_study_df = image_study_df[[""image_id"", ""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"train_df.drop(columns='Path', axis=1,inplace=T...","train_df.drop(columns='Path', axis=1,inplace=T...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"train_df.drop(columns=['Unnamed: 0'], inplace=...","train_df.drop(columns=['Unnamed: 0'], inplace=...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,study_df['PredictionString'] = study_df.progre...,study_df['PredictionString'] = study_df.progre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"df=df1[['SOPInstanceUID','PatientName','Catego...","df=df1[['SOPInstanceUID','PatientName','Catego...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"df = df[['id', 'PredictionString']]","df = df[['id', 'PredictionString']]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,train_im_dup = train_im_dup.drop(temp_train_im...,train_im_dup = train_im_dup.drop(temp_train_im...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,train_im_dup = train_im_dup.reset_index(drop=T...,train_im_dup = train_im_dup.reset_index(drop=T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df['promts'][0]

'BATCH_SIZE = 32\ntorch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)\ntorch_y_train ='

In [36]:
df['generated_text'][0]

'BATCH_SIZE = 32\ntorch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)\ntorch_y_train = torch.from_numpy(y_train).type(torch.LongTensor)\nmodel = Model(inputs=inputs, outputs=torch_X_train)\nmodel.compile(optimizer=Adam(0.001), loss=\'binary_crossentropy\', metrics=[\'accuracy\'])\nmodel.summary()\n\n\ndef get_validation_dataset(dataset, do_onehot=True):\n    dataset = dataset.batch(BATCH_SIZE)\n    dataset = dataset.prefetch(AUTO) \n    return dataset\ndef count_data_items(filenames):\n     \n   n = [int(re.compile(r"-([0-9]*)\\.").search(filename).group(1)) for filename in filenames]") for filename in filenames]\n      return np.sum(n)\ndef _get_train_datas_data(filenames, do_onehot=True):\n      \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n  

In [35]:
df['code_block'][0]

'BATCH_SIZE = 32\ntorch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)\ntorch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) \ntorch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)\ntorch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) \ntrain = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)\ntest = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)\ntrain_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)\ntest_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)\n'

In [12]:
df.to_csv(f'D:/Models/Logs/Label {Label}/Puntaje.csv', index=False)